In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import sys
import os
from PyQt4 import QtGui, QtCore

In [2]:
def adjustImage(img):
    
    #Crop image to make it square
    rows, cols, _ = img.shape
    min_axis = min(rows, cols)
    if rows != min_axis:
        diff = rows - min_axis
        dr = diff / 2
        img_new = img[dr:dr - diff, :, :]
    elif cols != min_axis:
        diff = cols - min_axis
        dc = diff / 2
        img_new = img[:, dc:dc - diff, :]
    
    #Resize to 499x499 pixels
    img_new = cv2.resize(img_new, (499, 499))
    
    #Add border to 500x500 pixels
    return cv2.copyMakeBorder(img_new, 1, 1, 1, 1, cv2.BORDER_CONSTANT, value = [0, 0, 0])

In [3]:
def countBacteria(img):
        
    img_bw = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    rows, cols = img_bw.shape
    
    #Removing noise and Otsu thresholding
    img_bw = cv2.GaussianBlur(img_bw, (5,5), 0)
    _, img_bw = cv2.threshold(img_bw, 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((3,3), np.uint8)
    img_bw = 1 - img_bw
    img_bw = cv2.morphologyEx(img_bw, cv2.MORPH_OPEN, kernel, iterations = 2)
    img_bw = 1 - img_bw
    
    #Finding Petri dish circle and removing outer part of the image
    xc = cols / 2 - 1
    yc = rows / 2 - 1
    radius = min(rows, cols) / 2
    prev = 0

    while True:

        circle = np.zeros(img_bw.shape, np.uint8)
        cv2.circle(circle, (xc,yc), radius, 1, 5)
        tot = np.sum(circle)
        img2 = np.logical_and(1 - img_bw, circle)
        curr = np.sum(img2) / float(tot)

        if (curr < prev and curr > 0.5):
            radius += 2
            break

        prev = curr
        radius -= 2

    circle = np.zeros(img_bw.shape, np.uint8)
    #cv2.circle(circle, (xc,yc), radius, 1, -1)
    cv2.circle(circle, (xc,yc), radius - 40, 1, -1)
    img_bw = np.logical_or(img_bw, 1 - circle).astype(np.uint8)
    
    kernel = np.ones((3,3), np.uint8)
    img_bw = 1 - img_bw
    img_bw = cv2.morphologyEx(img_bw, cv2.MORPH_OPEN, kernel, iterations = 2)
    img_bw = 1 - img_bw
    
    #Finding and counting single bacteria
    _, contours, _= cv2.findContours(img_bw, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    contours = contours[1:] #Removing the outer contour
    img_cont = img.copy()
    
    #Classifying bacteria
    singles = []
    clusters = []
    rejected = []

    for x in contours:
        rect = cv2.minAreaRect(x)
        if ((rect[1][0]*rect[1][1] > 0) and (0.8 < rect[1][0]/rect[1][0] < 1.2) and (0.7*np.pi/4 < cv2.contourArea(x)/(rect[1][0]*rect[1][1]) < 1.3*np.pi/4)):
            singles.append(x)
        else:
            rejected.append(x)
    
    mean = np.mean(map(cv2.contourArea, singles))
    singles = [x for x in singles if cv2.contourArea(x) > 0.3*mean]
    
    #Removing classified single contours from binary image
    img_bw = img_bw.astype(np.uint8)
    cv2.drawContours(img_bw, singles, -1, 1, -1)
    
    #Noise removal
    kernel = np.ones((3,3), np.uint8)
    img_bw = 255 * (1 - img_bw)
    opening = cv2.morphologyEx(img_bw, cv2.MORPH_OPEN, kernel, iterations = 2)

    #Sure background area
    sure_bg = cv2.dilate(opening, kernel, iterations=2)
    
    #Finding sure foreground area
    dt = cv2.distanceTransform(opening, cv2.DIST_L2, 5)
    dt = ((dt - dt.min()) / (dt.max() - dt.min()) * 255).astype(np.uint8)
    ret, sure_fg = cv2.threshold(dt, 100, 255, cv2.THRESH_BINARY)
    
    #Finding unknown region
    sure_fg = np.uint8(sure_fg)
    unknown = cv2.subtract(sure_bg, sure_fg)
    
    # Marker labelling
    ret, markers = cv2.connectedComponents(sure_fg)
    # Add one to all labels so that sure background is not 0, but 1
    markers = markers + 1
    # Now, mark the region of unknown with zero
    markers[unknown == 255] = 0
    
    markers = cv2.watershed(img, markers)
    
    for n in xrange(2, np.amax(markers) + 1):
        bac = np.zeros(img_bw.shape).astype(np.uint8)
        bac[markers == n] = 255
        
        _, cont, _ = cv2.findContours(bac, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        
        x = cont[0]
        rect = cv2.minAreaRect(x)
        if ((rect[1][0]*rect[1][1] > 0) and (0.8 < rect[1][0]/rect[1][0] < 1.2) and (0.8*np.pi/4 < cv2.contourArea(x)/(rect[1][0]*rect[1][1]) < 1.2*np.pi/4)):
            singles.append(x)
        else:
            rejected.append(x)
            
    img_fin = img.copy()
    cv2.drawContours(img_fin, singles, -1, (0, 255, 0), 3)
    
    #Return the total number of bacteria
    return len(singles), img_fin

In [5]:
class Example(QtGui.QWidget):
    
    def __init__(self):
        super(Example, self).__init__()
        
        self.initUI()
        
    def initUI(self):
        
        global countButton, lbl
        
        loadButton = QtGui.QPushButton("Load image...", self)
        loadButton.setMinimumWidth(200)
        loadButton.setMinimumHeight(50)
        countButton = QtGui.QPushButton("Count bacteria", self)
        countButton.setMinimumWidth(200)
        countButton.setMinimumHeight(50)
        countButton.setEnabled(False)
        
        loadButton.clicked.connect(self.loadClicked)
        countButton.clicked.connect(self.countClicked)
        
        lbl = QtGui.QLabel(self)
        pixmap = QtGui.QPixmap('noimage.png')
        lbl.setPixmap(pixmap)
        
        hbox = QtGui.QHBoxLayout()
        hbox.addStretch()
        hbox.addWidget(loadButton)
        hbox.addWidget(countButton)
        hbox.addStretch()

        vbox = QtGui.QVBoxLayout()
        vbox.addWidget(lbl)
        vbox.addStretch()
        vbox.addLayout(hbox)
        
        self.setLayout(vbox)
        
        self.setGeometry(200, 200, 500, 500)
        self.setWindowTitle('Bacterial Colony Counter')
        self.setWindowIcon(QtGui.QIcon('icon.png'))
        
        self.show()

    def loadClicked(self):
        
        global countButton, img, lbl, fname
        
        fname = QtGui.QFileDialog.getOpenFileName(self, 'Open file', 'c:\\', 'Image files (*.jpg *.jpeg *.png)')
        img = cv2.imread(str(fname))
        img_adj = adjustImage(img)
        qImg = QtGui.QImage(img_adj, img_adj.shape[1], img_adj.shape[0], img_adj.shape[1]*3, QtGui.QImage.Format_RGB888).rgbSwapped()
        lbl.setPixmap(QtGui.QPixmap(qImg))
        
        countButton.setEnabled(True)
        
    def countClicked(self):
        
        global img, lbl, tot, fname
        
        tot, img_fin = countBacteria(img)
        
        img_fin = adjustImage(img_fin)
        qImg = QtGui.QImage(img_fin, img_fin.shape[1], img_fin.shape[0], img_fin.shape[1]*3, QtGui.QImage.Format_RGB888).rgbSwapped()
        lbl.setPixmap(QtGui.QPixmap(qImg))
        
        message = "Number of bacteria = %d" %(tot)
        ans = QtGui.QMessageBox.information(self, "Information", message, QtGui.QMessageBox.Save, QtGui.QMessageBox.Close)
        
        if ans == QtGui.QMessageBox.Save:
            with open("LOG.txt", "a") as my_file:
                my_file.write("%s\t%d\n" %(fname, tot))

            if not my_file.closed:
                my_file.close()

def main():
    
    app = QtGui.QApplication(sys.argv)
    ex = Example()
    sys.exit(app.exec_())


if __name__ == '__main__':
    main()

SystemExit: 0